In [2]:
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image

import numpy as np
import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


####  1. Content image, C => image to put style on
####  2. Style image, S
####  3. Generated image, G => Content image with style of S

Using pretrained model, going to take activation of hidden layer somewhere in the middle of the network as result. And Using custom Cost function going to tune the network, to create a C image with style of S, image G **Using aG activation of dimension (1, nH, nW, nC).

# 1. Defining Cost for the neural netwrok

1. Content cost, JC
2. Style cost, SC
3. Combining the two costs, J

## 1.1 Content Cost, JC

Content cost takes a hidden layer activation for both C and G images, aC and aG, and measure their difference. So, when we minimize this cost, it will make Genrated image G contents similar to C image.

In [2]:
def contentCost(aC, aG):
    """
    Computes the content cost
    
    Arguments:
    aC -- tensor of dimension (1, nH, nW, nC)
    aG -- tensor of dimension (1, nH, nW, nC)
    
    Returns: 
    JC -- cost for the content image
    """
    
    # Retrieve dimensions from a_G
    m, nH, nW, nC = aG.get_shape().as_list()
    
    # Reshape aC and aG
    aC_unrolled = tf.reshape(aC, [m, nH * nW, nC])
    aG_unrolled = tf.reshape(aG, [m, nH * nW, nC])
    
    
    JC = (1 / (4 * nH * nW * nC)) * tf.reduce_sum(tf.square(tf.subtract(aC_unrolled, aG_unrolled)))
        
    return JC

## 1.2 Style Cost, JS

Gram matrix G of a set of vectors (v1,…,vn) is the matrix of dot products

Gij = np.dot(vi,vj)

large Gij means vi is very similar to vj

In [1]:
def gramMatrix(A):
    """
    Argument:
    A -- matrix of shape (nC, nH * nW)
    
    Returns:
    GM -- Gram matrix of A, of shape (nC, nC)
    """
    
    GM = tf.matmul(A, A, transpose_b=True)
    
    return GM

need to minimize the distance between the Gram matrix of the style image S and that of the generated image G.

In [6]:
def styleCost(aS, aG):
    """
    Arguments:
    aS -- tensor of dimension (1, nH, nW, nC)
    aG -- tensor of dimension (1, nH, nW, nC)
    
    Returns: 
    JS -- scalar style cost
    """
    
    m, nH, nW, nC = aG.get_shape().as_list()
    
    # Reshape the images
    # if m images, do it using a for loop
    aS = tf.transpose(tf.reshape(aS, [nH * nW, nC]))
    aG = tf.transpose(tf.reshape(aG, [nH * nW, nC]))

    # Computing gram_matrices for S and G
    GMS = gramMatrix(aS)
    GMG = gramMatrix(aG)
    
    
    # Computing the loss (≈1 line)
    JS = (1 / ((2 * nC * nH * nW) ** 2)) * tf.reduce_sum(tf.square(tf.subtract(GMS, GMG)))
        
    return JS

In [7]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(2)
    aS = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    aG = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    JS = styleCost(aS, aG)
    
    print("JS = " + str(JS.eval()))

NameError: name 'gram_matrix' is not defined